In [1]:
from hedge_stand import HedgeST,HedgeSTBoll
from datetime import date, datetime
from demeter import TokenInfo, PoolBaseInfo, Actuator, Strategy, Asset, AccountStatus, BuyAction, SellAction, RowData, \
    ChainType
import optunity
import optunity.metrics
from decimal import Decimal
import pandas as pd
import  matplotlib.pylab as plt
# import logging 
# from logging import handlers
from load_data import pool_id_1_eth_u_500

from strategy_ploter import  plot_position_return_decomposition

import quantstats as qs
qs.extend_pandas()

%matplotlib inline

def backtest_boll(a, hedge_spread_split,hedge_spread_rate,alpha,boll_period_n):
    # global RUNNING_TIME
    # print(f"==================spread running time {RUNNING_TIME}==================")

    decimal_a = Decimal(a).quantize(Decimal('0.00'))
    decimal_hedge_spread_split = Decimal(hedge_spread_split).quantize(Decimal('0.0'))
    decimal_hedge_spread_rate = Decimal(hedge_spread_rate).quantize(Decimal('0.00'))

    ema_max_spread_rate = round(0.2,4)

    eth = TokenInfo(name="eth", decimal=18)
    usdc = TokenInfo(name="usdc", decimal=6)
    pool = PoolBaseInfo(usdc, eth, 0.05, usdc)

    #收益计算基础参数
    # net_value_base = 'ETH'

    runner_instance = Actuator(pool)
    # runner_instance.enable_notify = False
    runner_instance.strategy = HedgeSTBoll(a=decimal_a,hedge_spread_split = decimal_hedge_spread_split,hedge_spread_rate=decimal_hedge_spread_rate,alpha=alpha,ema_max_spread_rate=ema_max_spread_rate,period_n=int(boll_period_n))
    runner_instance.set_assets([Asset(usdc, 10000)])
    save_path = f"../demeter/data/ETH/{pool_id_1_eth_u_500}"
    runner_instance.data_path = save_path
    runner_instance.load_data(ChainType.Ethereum.name,
                                pool_id_1_eth_u_500,
                                DATE_START,
                               DATE_END)
    runner_instance.run(enable_notify=False)

    hedge_count = runner_instance.strategy.hedge_count
    outside_ema_count = runner_instance.strategy.outside_ema_count

    # df_status = pd.DataFrame(runner_instance.account_status_list)

    total_net_value = runner_instance.final_status.net_value
    
    final_total_usdc_value = round(total_net_value + runner_instance.strategy.e.df['total'].iloc[-1],3)
    
    final_price = runner_instance.final_status.price

    final_total_eth_value = round(final_total_usdc_value / final_price,3)

    notice = f"ema+alpha:{RUNNING_TIME} times, a:{decimal_a}, hedge_spread_split:{decimal_hedge_spread_split}, hedge_spread_rate:{decimal_hedge_spread_rate},alpa:{alpha}"
    result =f" result: outside_ema_count:{outside_ema_count},hedge count:{hedge_count} final_total_eth_value:{final_total_eth_value},final_total_usdc_value:{final_total_usdc_value}"  
    print(notice)
    print(result)
    # if SEND_NOTICE:
    #     send_notice('CEX_Notify',notice)

    # RUNNING_TIME +=1

    # if NET_VALUE_BASE == 'USDC':
    #     print(final_total_usdc_value)
    #     return float(final_total_usdc_value)
    #     # profit_rate_usdc = profit_usdc / runner_instance.strategy.init_total_usdc
    # else:
    #     print(float(final_total_usdc_value / final_price))
    #     return float(final_total_usdc_value / final_price)


    return runner_instance


def backtest_instance(a, hedge_spread_split,hedge_spread_rate,alpha):
    # global RUNNING_TIME
    # print(f"==================spread running time {RUNNING_TIME}==================")

    decimal_a = Decimal(a).quantize(Decimal('0.00'))
    decimal_hedge_spread_split = Decimal(hedge_spread_split).quantize(Decimal('0.0'))
    decimal_hedge_spread_rate = Decimal(hedge_spread_rate).quantize(Decimal('0.00'))

    alpha = round(alpha,4)
    # ema_max_spread_rate = round(ema_max_spread_rate,4)

    eth = TokenInfo(name="eth", decimal=18)
    usdc = TokenInfo(name="usdc", decimal=6)
    pool = PoolBaseInfo(usdc, eth, 0.05, usdc)

    #收益计算基础参数
    # net_value_base = 'ETH'

    runner_instance = Actuator(pool)
    # runner_instance.enable_notify = False
    runner_instance.strategy = HedgeST(decimal_a,decimal_hedge_spread_split,decimal_hedge_spread_rate,alpha)
    runner_instance.set_assets([Asset(usdc, 10000)])
    save_path = f"../demeter/data/ETH/{pool_id_1_eth_u_500}"
    runner_instance.data_path = save_path
    runner_instance.load_data(ChainType.Ethereum.name,
                                pool_id_1_eth_u_500,
                                DATE_START,
                               DATE_END)
    runner_instance.run(enable_notify=False)

    hedge_count = runner_instance.strategy.hedge_count
    outside_ema_count = runner_instance.strategy.outside_ema_count

    # df_status = pd.DataFrame(runner_instance.account_status_list)

    total_net_value = runner_instance.final_status.net_value
    
    final_total_usdc_value = round(total_net_value + runner_instance.strategy.e.df['total'].iloc[-1],3)
    
    final_price = runner_instance.final_status.price

    final_total_eth_value = round(final_total_usdc_value / final_price,3)

    #处理df_status
    df_status = pd.DataFrame(runner_instance.account_status_list)
    df_merge = pd.merge(e.df, df_status, on='timestamp', how='inner')

    df_merge['total_value'] = df_merge['total'] + df_merge['net_value']
    df_merge['total_value_eth'] = df_merge['total_value'] / df_merge['price']

    df_merge.set_index('timestamp', inplace=True)




    notice = f"ema+alpha:{RUNNING_TIME} times, a:{decimal_a}, hedge_spread_split:{decimal_hedge_spread_split}, hedge_spread_rate:{decimal_hedge_spread_rate},alpa:{alpha}"
    result =f" result: outside_ema_count:{outside_ema_count},hedge count:{hedge_count} final_total_eth_value:{final_total_eth_value},final_total_usdc_value:{final_total_usdc_value}"  
    print(notice)
    print(result)
    # if SEND_NOTICE:
    #     send_notice('CEX_Notify',notice)

    # RUNNING_TIME +=1

    # if NET_VALUE_BASE == 'USDC':
    #     print(final_total_usdc_value)
    #     return float(final_total_usdc_value)
    #     # profit_rate_usdc = profit_usdc / runner_instance.strategy.init_total_usdc
    # else:
    #     print(float(final_total_usdc_value / final_price))
    #     return float(final_total_usdc_value / final_price)


    return runner_instance





In [2]:

def generate_dfmerge(runner_instance):
    e = runner_instance.strategy.e

    total_net_value = runner_instance.final_status.net_value
        
    final_total_usdc_value = total_net_value + runner_instance.strategy.e.df['total'].iloc[-1]


    final_price = runner_instance.final_status.price
    # if net_value_base == 'USDC':
    profit_usdc = round(final_total_usdc_value- runner_instance.strategy.init_total_usdc,3)
    profit_rate_usdc = round(profit_usdc / runner_instance.strategy.init_total_usdc,3)
    # else:
    profit_eth = round(final_total_usdc_value/final_price - runner_instance.strategy.init_total_symbol,3)
    profit_rate_eth = round(profit_eth / runner_instance.strategy.init_total_symbol,3)

    print(f"USDC profit: {profit_usdc}, profit_rate: {profit_rate_usdc}")

    print(f"ETH profit: {profit_eth}, profit_rate: {profit_rate_eth}")

    df_status = pd.DataFrame(runner_instance.account_status_list)
    df_merge = pd.merge(e.df, df_status, on='timestamp', how='inner')

    df_merge['total_value'] = df_merge['total'] + df_merge['net_value']

    df_merge.set_index('timestamp', inplace=True)

    df_merge['total_value_profit_rate'] = df_merge['total_value'] / runner_instance.strategy.init_total_usdc


    df_merge['total_value_eth'] = df_merge['total_value'] / df_merge['price']

    df_merge['net_value_eth'] = df_merge['net_value'] / df_merge['price']

    df_merge['total_value_eth_profit_rate'] = df_merge['total_value_eth'] / runner_instance.strategy.init_total_symbol

    df_merge_f = df_merge.apply(pd.to_numeric, downcast='float')
   
    df_merge_f['net_value_rate'] = df_merge_f['net_value'] / df_merge_f['net_value'].iloc[0]

    df_merge_f['price_rate'] = df_merge_f['price'] / df_merge_f['price'].iloc[0]
    return df_merge_f

In [3]:
# pd.options.plotting.backend = "plotly"

CHAIN_ID = 'ETH'
NET_VALUE_BASE = 'ETH'
RUNNING_TIME = 1
SEND_NOTICE = False
str_date_start = '2022-8-1'
str_date_end = '2022-10-31'

DATE_START = datetime.strptime(str_date_start, "%Y-%m-%d").date()

DATE_END = datetime.strptime(str_date_end, "%Y-%m-%d").date()

# A1 test

In [4]:
a = 1.2
hedge_spread_split = 2.5
hedge_spread_rate = 0.91
alpha = 0.037
ema_max_spread_rate=0.2
period_n = 17

instance=backtest_boll(a=a,hedge_spread_split=hedge_spread_split,hedge_spread_rate=hedge_spread_rate,alpha=alpha,boll_period_n=period_n)

df_merge = generate_dfmerge(instance)

# fig = df_merge[['price_rate','total_value_eth_profit_rate','total_value_profit_rate']].plot().get_figure()

# fig.savefig('./fig/test.pdf')

# plot = df_merge[['price_rate','total_value_eth_profit_rate','net_value_rate','total_value_profit_rate']].plot()

# plot.write_image(f"./fig/{NET_VALUE_BASE}_{DATE_START}_{DATE_END}_{a}_{hedge_spread_split}_{hedge_spread_rate}_{alpha}_{period_n}.png")

# plot.show()

2022-12-04 23:33:02,009 - INFO - start load files from 2022-08-01 to 2022-10-31...


init parameters: a:1.20, hedge_spread_split:2.5, hedge_spread_rate:0.91,alpa:0.037,period_n:17



2022-12-04 23:33:03,835 - INFO - load file complete, preparing...
2022-12-04 23:33:09,617 - INFO - data has benn prepared
2022-12-04 23:33:09,618 - INFO - init strategy...


add ema column with alpha 0.037
132480


2022-12-04 23:33:12,031 - INFO - start main loop...


...

2022-10-25 18:29:00 hedge up buy ETH, c_amount:2.283065106624601760353976227,f_amount:3.491155133706169786058879428,upper:1353.97,middle:1278.44,lower:,1202.91, trade price:1509.202558299111363811045646,ema:1475.4496039888609, trade amount:1.084978728242633676559242548, hedge count:2


 94%|██████████████████████████████████████████████████████████████████████████████████████████████████▊      | 124726/132480 [48:55<02:51, 45.20it/s]

2022-10-26 14:41:00 hedge up buy ETH, c_amount:1.209339071440708409581658223,f_amount:4.576133861948803462618121976,upper:1401.81,middle:1280.86,lower:,1159.91, trade price:1586.101183993584895229706538,ema:1560.5061969669125, trade amount:1.084978728242633676559242548, hedge count:3


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 132480/132480 [51:54<00:00, 42.54it/s]
2022-12-05 00:25:06,453 - INFO - main loop finished, start calculate evaluating indicator...
2022-12-05 00:25:07,551 - INFO - run evaluating indicator
2022-12-05 00:25:07,660 - INFO - back testing finish


ema+alpha:1 times, a:1.20, hedge_spread_split:2.5, hedge_spread_rate:0.91,alpa:0.037
 result: outside_ema_count:0,hedge count:3 final_total_eth_value:8.069,final_total_usdc_value:12698.707
USDC profit: 698.707, profit_rate: 0.058
ETH profit: 0.915, profit_rate: 0.128


In [17]:


filename = f"{NET_VALUE_BASE}_{DATE_START}_{DATE_END}_{a}_{hedge_spread_split}_{hedge_spread_rate}_{alpha}_{period_n}.html"
# df_merge['total_value'].plot_snapshot()
qs.reports.html(df_merge['total_value_eth'],benchmark=df_merge['net_value_eth'],mode='full',periods_per_year=365, download_filename=f"./reports/{filename}",output='report')


In [11]:
df_merge

,margin,total,leverage,realised_profit,unrealised_profit,base_balance,quote_balance,base_uncollected,quote_uncollected,base_in_position,quote_in_position,net_value,price,total_value,total_value_profit_rate,total_value_eth,net_value_eth,total_value_eth_profit_rate,net_value_rate,price_rate
timestamp,,,,,,,,,,,,,,,,,,,,
2022-08-01 00:00:00,249.937500,1996.250977,2.504,-3.749063,0.000000,0.118199,0.0,0.000063,0.000013,4997.381836,2.980711,9997.522461,1677.452271,11993.772461,0.999481,7.149994,5.959944,0.999481,1.000000,1.000000
2022-08-01 00:01:00,249.937500,2004.755615,2.494,-3.749063,8.504677,0.118199,0.0,0.043984,0.000024,5046.160645,2.951656,10006.048828,1680.306152,12010.804688,1.000900,7.147986,5.954896,0.999200,1.000853,1.001701
2022-08-01 00:02:00,249.937500,1999.250977,2.500,-3.749063,3.000000,0.118199,0.0,0.051132,0.000037,5014.593262,2.970454,10000.609375,1678.458984,11999.860352,0.999988,7.149332,5.958209,0.999389,1.000309,1.000600
2022-08-01 00:03:00,249.937500,1995.751099,2.504,-3.749063,-0.499825,0.118199,0.0,0.051664,0.000044,4994.513672,2.982421,9997.125977,1677.284546,11992.876953,0.999406,7.150175,5.960304,0.999506,0.999960,0.999900
2022-08-01 00:04:00,249.937500,1998.250732,2.502,-3.749063,1.999800,0.118199,0.0,0.063054,0.000052,5008.855469,2.973872,9999.649414,1678.123291,11997.899414,0.999825,7.149594,5.958828,0.999425,1.000213,1.000400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-31 23:55:00,410.159912,1835.486694,4.470,-876.889709,712.376404,0.392371,0.0,354.724731,0.260337,8473.352539,1.036530,10870.881836,1574.880249,12706.368164,1.058864,8.068149,6.902672,1.127828,1.087358,0.938852
2022-10-31 23:56:00,410.159912,1829.248291,4.484,-876.889709,706.138000,0.392371,0.0,354.724762,0.260345,8452.627930,1.049695,10869.457031,1573.778320,12698.705078,1.058225,8.068929,6.906600,1.127937,1.087215,0.938196
2022-10-31 23:57:00,410.159912,1829.248291,4.484,-876.889709,706.138000,0.392371,0.0,354.726166,0.260345,8452.627930,1.049695,10869.458008,1573.778320,12698.706055,1.058226,8.068930,6.906600,1.127937,1.087215,0.938196


# A2 test

In [16]:
df_merge['total_value_eth'].sharpe(benchmark=df_merge['net_value_eth'])

TypeError: sharpe() got an unexpected keyword argument 'benchmark'

In [6]:
a = 1.20
hedge_spread_split = 2.5
hedge_spread_rate = 0.86
alpha = 0.037
ema_max_spread_rate=0.2
period_n2 = 9

instance2=backtest_boll(a=a,hedge_spread_split=hedge_spread_split,hedge_spread_rate=hedge_spread_rate,alpha=alpha,boll_period_n=period_n2)

df_merge2 = generate_dfmerge(instance2)

# fig = df_merge[['price_rate','total_value_eth_profit_rate','total_value_profit_rate']].plot().get_figure()

# fig.savefig('./fig/test.pdf')

# plot2 = df_merge2[['price_rate','total_value_eth_profit_rate','net_value_rate','total_value_profit_rate']].plot()

# plot2.write_image(f"./fig/{NET_VALUE_BASE}_{DATE_START}_{DATE_END}_{a}_{hedge_spread_split}_{hedge_spread_rate}_{alpha}_{period_n2}.png")

# plot2.show()
filename = f"{NET_VALUE_BASE}_{DATE_START}_{DATE_END}_{a}_{hedge_spread_split}_{hedge_spread_rate}_{alpha}_{period_n2}.html"
# df_merge['total_value'].plot_snapshot()
qs.reports.html(df_merge2['total_value_eth'],benchmark=df_merge2['net_value_eth'],mode='full',periods_per_year=365, download_filename=f"./reports/{filename}",output='report')

2022-12-05 00:26:17,092 - INFO - start load files from 2022-08-01 to 2022-10-31...


init parameters: a:1.20, hedge_spread_split:2.5, hedge_spread_rate:0.86,alpa:0.037,period_n:9



2022-12-05 00:26:19,942 - INFO - load file complete, preparing...
2022-12-05 00:26:28,314 - INFO - data has benn prepared
2022-12-05 00:26:28,315 - INFO - init strategy...


add ema column with alpha 0.037
132480


2022-12-05 00:26:30,488 - INFO - start main loop...


...

2022-10-25 18:29:00 hedge up buy ETH, c_amount:2.283065106624601760353976227,f_amount:3.491155133706169786058879428,upper:1344.72,middle:1301.25,lower:,1257.79, trade price:1509.202558299111363811045646,ema:1475.4496039888609, trade amount:1.025364512405126331693350100, hedge count:11


 94%|██████████████████████████████████████████████████████████████████████████████████████████████████▊      | 124727/132480 [23:24<01:55, 67.27it/s]

2022-10-26 14:35:00 hedge up buy ETH, c_amount:1.266435217476170181445245601,f_amount:4.516519646111296117752229528,upper:1414.14,middle:1306.67,lower:,1199.20, trade price:1573.148978498663393278718011,ema:1555.8002083821084, trade amount:1.025364512405126331693350100, hedge count:12


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 132480/132480 [25:28<00:00, 86.65it/s]
2022-12-05 00:51:59,418 - INFO - main loop finished, start calculate evaluating indicator...
2022-12-05 00:51:59,646 - INFO - run evaluating indicator
2022-12-05 00:51:59,698 - INFO - back testing finish


ema+alpha:1 times, a:1.20, hedge_spread_split:2.5, hedge_spread_rate:0.86,alpa:0.037
 result: outside_ema_count:0,hedge count:12 final_total_eth_value:8.009,final_total_usdc_value:12603.841
USDC profit: 603.841, profit_rate: 0.050
ETH profit: 0.855, profit_rate: 0.120


# a3 ema hedge

In [7]:
a = 1.20
hedge_spread_split = 2.5
hedge_spread_rate = 0.86
alpha3 = 0.037
ema_max_spread_rate2=0.2

instance3=backtest_instance(a,hedge_spread_split,hedge_spread_rate,alpha3)

df_merge3 = generate_dfmerge(instance3)

filename = f"{NET_VALUE_BASE}_{DATE_START}_{DATE_END}_{a}_{hedge_spread_split}_{hedge_spread_rate}_{alpha}.html"
# df_merge['total_value'].plot_snapshot()
qs.reports.html(df_merge['total_value'],benchmark=df_merge['price'],mode='full',periods_per_year=365, download_filename=f"./reports/{filename}",output='report')

# plot3 = df_merge3[['price_rate','total_value_eth_profit_rate','net_value_rate','total_value_profit_rate']].plot()

# plot3.write_image(f"./fig/{NET_VALUE_BASE}_{DATE_START}_{DATE_END}_{a}_{hedge_spread_split}_{hedge_spread_rate}_{alpha3}.png")

# plot3.show()
filename = f"{NET_VALUE_BASE}_{DATE_START}_{DATE_END}_{a}_{hedge_spread_split}_{hedge_spread_rate}_{alpha3}.html"
# df_merge['total_value'].plot_snapshot()
qs.reports.html(df_merge3['total_value_eth'],benchmark=df_merge3['net_value_eth'],mode='full',periods_per_year=365, download_filename=f"./reports/{filename}",output='report')

2022-12-05 00:53:07,682 - INFO - start load files from 2022-08-01 to 2022-10-31...


init parameters: a:1.20, hedge_spread_split:2.5, hedge_spread_rate:0.86,alpa:0.037,period_n:1440



2022-12-05 00:53:09,285 - INFO - load file complete, preparing...
2022-12-05 00:53:13,792 - INFO - data has benn prepared
2022-12-05 00:53:13,792 - INFO - init strategy...


add ema column with alpha 0.037
132480


2022-12-05 00:53:15,150 - INFO - start main loop...


...

2022-10-26 07:30:00 hedge buy ETH, trade price:1515.705776042607361151642602,ema:1499.2020278486564, trade amount:1.025364512405126331693350100, hedge count:22


 97%|█████████████████████████████████████████████████████████████████████████████████████████████████████▊   | 128395/132480 [19:45<01:03, 64.05it/s]

2022-10-29 03:46:00 hedge buy ETH, trade price:1587.053082650883048131393992,ema:1582.1192788869112, trade amount:1.025364512405126331693350100, hedge count:23


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 132480/132480 [20:51<00:00, 105.84it/s]
2022-12-05 01:14:06,804 - INFO - main loop finished, start calculate evaluating indicator...
2022-12-05 01:14:07,527 - INFO - run evaluating indicator
2022-12-05 01:14:07,601 - INFO - back testing finish


ema+alpha:1 times, a:1.20, hedge_spread_split:2.5, hedge_spread_rate:0.86,alpa:0.037
 result: outside_ema_count:0,hedge count:23 final_total_eth_value:7.360,final_total_usdc_value:11582.853
USDC profit: -417.147, profit_rate: -0.035
ETH profit: 0.206, profit_rate: 0.029
